In [1]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

In [2]:
# For nocaps:
annFile='../Datasets/NoCaps/nocaps_val_4500_captions.json'
resFile = "../knowledge_induced_decoder/generated_sentences/vit_KG_pll_base_enc1_pttok_rc3_coco_beam_1_Nocaps.json"

In [3]:
# result for coco
annotation_file = 'captions_val2014.json'
# results_file = 'captions_val2014_fakecap_results.json'
# results_file = "../../generated_sentences/vit_KG_pll_base_enc1_pttok_rc3_coco_beam_1.json"
results_file = "../../generated_sentences/vit_KG_vanil_base_enc1_final_org_SegParam2_coco.json"



In [4]:
datatype = "coco"
# model_file = "../../generated_sentences/vit_kw_pll_base_enc1_final_org_SegParam2"
model_file = "../../generated_sentences/scst_gen/vit_KG_pll_base_enc1_final_org_SegParam2"

if datatype == "coco":
    annotation_file = 'captions_val2014.json'
else:
    annotation_file='../../../Datasets/NoCaps/nocaps_val_4500_captions.json'
results_file = model_file + "_" +datatype + ".json"

In [5]:
# %pip install tabulate
import os.path
from os import path
import json
from tabulate import tabulate
def load_json(filename):
    "Wrapper function to load JSON data."
    with open(filename) as f:
        data = json.load(f)
    return data

In [6]:
annotation_file = 'captions_val2014.json'
import numpy as np
from glob import glob
all_capfiles = glob("../../generated_sentences/*.json", recursive = True)
all_capfiles.sort()
print(len([file for file in all_capfiles if "coco" in file]))
basefile = "../../generated_sentences/vit_KG_pll_base_enc1_final_org_SegParam2_coco.json".split("_")
basename = "axolotl"
metric_names = ["Bleu_1", "Bleu_4", "METEOR", "ROUGE_L", "CIDEr", "SPICE"]
table = [["model_name"] +metric_names]
for file in all_capfiles:
    if "coco" in file:
#         print(file)
        splitfile = file.split("_")
        difffile = [word for word in splitfile if word not in basefile]
        newname = basename+"_" +"_".join(difffile)
        print(newname)
        coco = COCO(annotation_file)
        coco_result = coco.loadRes(file)
        coco_eval = COCOEvalCap(coco, coco_result)
        coco_eval.params['image_id'] = coco_result.getImgIds()
        coco_eval.evaluate()
        entry = [newname]
        model_res = list(np.array([coco_eval.eval[metr] for metr in metric_names])*100)
        entry += model_res
        print(model_res)
        table.append(entry)

16
axolotl_sentences/promptsets
loading annotations into memory...
Done (t=0.29s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.10s)
creating index...


KeyError: 'id'

In [10]:
print(tabulate(table, headers='firstrow'))

model_name             Bleu_1    Bleu_4    METEOR    ROUGE_L    CIDEr    SPICE
-------------------  --------  --------  --------  ---------  -------  -------
axolotl_NoRel         76.9148   35.5604   27.1589    56.4772  113.648  20.1768
axolotl_RCl2r         75.9093   35.177    27.0816    56.0693  112.945  20.2326
axolotl_SEGKW         76.8176   35.5387   27.2002    56.467   113.349  20.2482
axolotl_              81.7776   37.2962   27.5949    57.3121  121.197  20.5645
axolotl_plldec1       76.4629   34.831    27.2973    56.4239  112.645  20.3951
axolotl_posidx1       76.29     34.9314   27.0813    56.2933  112.974  20.2206
axolotl_wo_pttokemb   76.6191   35.4325   27.151     56.5817  113.404  20.282
axolotl_stck          76.6666   35.6669   27.0801    56.432   112.85   20.2195
axolotl_kw            80.9396   36.0857   27.5111    56.8609  119.802  20.4818


In [1]:
# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)
coco_eval = COCOEvalCap(coco, coco_result)
coco_eval.params['image_id'] = coco_result.getImgIds()
coco_eval.evaluate()

In [ ]:
# print output evaluation scores
for metric, score in coco_eval.eval.items():
    print(f'{metric}: {score:.3f}')